# Python для сбора и анализа данных

*Алла Тамбовцева*

## Практикум 7. Парсинг с `BeautifulSoup`: тэги и атрибуты

Мы уже немного познакомились со структурой html-страниц и поиском информации по тэгам, теперь попробуем выгрузить информацию из более интересной страницы, а точнее, с сайта [nplus1.ru](https://nplus1.ru/). Наша задача – выгрузить недавние новости в датафрейм `pandas`, чтобы потом сохранить все в файл Excel.

Для работы нам снова понадобится модуль `requests` для отправки запросов, для «подключения» к странице и получения ее содержимого в виде строки, и функция `BeautifulSoup` из библиотеки `bs4` для удобного поиска по полученной строке:

In [1]:
import requests
from bs4 import BeautifulSoup

Сохраним ссылку на главную страницу в переменную `main` и отправим запрос к ней с помощью функции `get()` из `requests`:

In [2]:
main = "https://nplus1.ru/"
page = requests.get(main)

In [3]:
page

<Response [200]>

Заберём исходный код страницы и преобразуем строку с ним в объект `BeautifulSoup`:

In [ ]:
# обычная строка с исходным кодом

page.text

In [5]:
soup = BeautifulSoup(page.text)

Чтобы сгрузить все новости с главной страницы сайта, нужно собрать все ссылки на страницы с этими новостями. Ссылки в html-файле всегда заключены в тэг `<a></a>` и имеют атрибут `href`. Найдем кусочки кода HTML, соответствующие всем ссылкам на главной странице сайта:

In [6]:
links_raw = soup.find_all("a") 
links_raw[10:20]  # несколько штук для примера

[<a class="hover:underline transition-colors duration-75" href="/search/empty/768">Генетика</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/890">Математика</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/871">Космонавтика</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/876">Археология</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/775">Нейронауки</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/767">На мышах</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/771">Звук</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/772">Красота</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/778">Научные закрытия</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/917">ИИ спешит на помощь</a>]

Каждый элемент возвращаемого списка имеет тип `BeautifulSoup` и структуру, очень похожую на словарь. Например, ссылка `<a class="hover:underline transition-colors duration-75" href="/search/empty/768">Генетика</a>` изнутри выглядит как словарь следующего вида:

    {'href' : '/search/empty/768', 
     'class' : 'hover:underline transition-colors duration-75'}.
    
Как мы помним, значение по ключу из словаря можно вызвать с помощью метода `.get()`. Давайте извлечем значения по ключу `href` из каждого элемента списка `links`:

In [7]:
links = [li.get("href") for li in links_raw] 
links[10:20]  # несколько штук для примера

['/search/empty/768',
 '/search/empty/890',
 '/search/empty/871',
 '/search/empty/876',
 '/search/empty/775',
 '/search/empty/767',
 '/search/empty/771',
 '/search/empty/772',
 '/search/empty/778',
 '/search/empty/917']

Ссылок в списке выше много. Но нам нужны только новости – ссылки, которые начинаются с `https://nplus1.ru/news`. Создадим пустой список `news` и будем добавлять в него только ссылки, которые удовлетворяют этому условию.

In [8]:
news = []
for li in links:
    if "https://nplus1.ru/news/" in li:
        news.append(li)

In [9]:
news[0:10]

['https://nplus1.ru/news/2015/09/21/editor-thy-name',
 'https://nplus1.ru/news/2023/04/06/pseudorabdion-longiceps',
 'https://nplus1.ru/news/2023/04/06/in-abhorrence-dementia',
 'https://nplus1.ru/news/2023/04/06/sun-is-sun',
 'https://nplus1.ru/news/2023/04/06/RSVpreF',
 'https://nplus1.ru/news/2023/04/06/muons-Napoli',
 'https://nplus1.ru/news/2023/04/06/pseudorabdion-longiceps',
 'https://nplus1.ru/news/2023/04/05/double-quasar-confirmed',
 'https://nplus1.ru/news/2023/04/05/push-jab',
 'https://nplus1.ru/news/2023/04/05/yz-ceti']

Первая ссылка ведет не совсем на новость, скорее, на объявление, поэтому давайте ее уберем:

In [10]:
news = news[1:] 

Теперь наша задача сводится к следующему: изучить одну страницу с новостью, научиться из нее вытаскивать текст и всю необходимую информацию, а потом применить весь набор действий к каждой ссылке из `news` в цикле. Посмотрим на новость с индексом 0, у вас может быть другая, новости обновляются.

In [11]:
link0 = news[0]
print(link0)

https://nplus1.ru/news/2023/04/06/pseudorabdion-longiceps


### Задача 1

Отправьте запрос к странице по ссылке `link0` с одной новостью, получите результат в виде объекта `BeautifulSoup` и сохраните его как `soup0`.

In [12]:
page0 = requests.get(link0)
soup0 = BeautifulSoup(page0.text)

### Задача 2

Найдите заголовок новости и сохраните его в переменную `title`.

In [15]:
title = soup0.find("title").text

### Задача 3

Найдите имя автора новости и дату её публикации. Сохраните их в `author` и `date` соответственно.

In [20]:
date = soup0.find("meta", {"itemprop" : "datePublished"}).get("content") 
author = soup0.find("meta", {"name" : "author"}).get("content") 
print(date, author)

2023-04-06 Сергей Коленов


### Задача 4

Найдите сложность новости и рубрики, к которым она относится. Сохраните сложность в переменную `diffc`. Рубрики сначала можно сохранить в список, а затем его элементы объединить в одну строку `rubs`.

**Подсказка:** чтобы упростить себе жизнь, найдите сначала раздел (`div`), в котором на странице хранится дата и время публикации, сложность и рубрики, а затем выполняйте поиск в рамках этого раздела.

In [29]:
div = soup0.find("div", 
                 {"class" : "flex flex-wrap lg:mb-10 gap-2 text-tags xl:pr-9"})
diffc = div.find_all("span")[3].text
rubs_raw = div.find_all("span")[4:] 
rubs_L = [r.text for r in rubs_raw] 
rubs = ", ".join(rubs_L)
rubs

'Зоология'

### Задача 5

Соберите из абзацев текст новости и сохраните его в переменную `text`. Избавьтесь от постронних символов (`\xa0`, `\n`) в тексте. 

In [32]:
pars_raw = soup0.find_all("p", {"class" : "mb-6"}) 
pars = [p.text for p in pars_raw] 
text = " ".join(pars)
text = text.replace("\xa0", " ").replace("\n", " ")
text

'Такой защитный механизм описан у рептилий впервые Герпетологи описали новый способ передвижения у змей. Они обнаружили, что ужеобразная змея Pseudorabdion longiceps в случае опасности способна катиться колесом, активно отталкиваясь от поверхности земли. Так она не только быстрее убегает от хищника, но и сбивает его с толку. Похожим образом спасаются от врагов многие беспозвоночные и некоторые амфибии, однако у рептилий он прежде известен не был. Результаты исследования опубликованы в статье для журнала Biotropica. Змеи выработали много разнообразных способов защищаться от хищников. Некоторые из них в случае опасности просто уползают, другие полагаются на маскировочную окраску, а третьи принимают угрожающую позу и издают предупреждающие звуки, например, шипят или трещат погремушкой на хвосте. А в трех группах кобр независимо появились плюющиеся виды, которые отгоняют врагов, стреляя им ядом в глаза, причем яд этих змей причиняет намного более сильную боль, чем яд обычных кобр. Команда 

### Задача 6

Напишите функцию `get_news()`, которая принимает на вход ссылку на страницу с одной новостью, а возвращает список из следующих характеристик: имя автора, дата публикации, сложность новости, рубрики, текст новости.

In [33]:
def get_news(link0):
    page0 = requests.get(link0)
    soup0 = BeautifulSoup(page0.text)
    title = soup0.find("title").text
    date = soup0.find("meta", {"itemprop" : "datePublished"}).get("content") 
    author = soup0.find("meta", {"name" : "author"}).get("content") 
    div = soup0.find("div", 
                 {"class" : "flex flex-wrap lg:mb-10 gap-2 text-tags xl:pr-9"})
    diffc = div.find_all("span")[3].text
    rubs_raw = div.find_all("span")[4:] 
    rubs_L = [r.text for r in rubs_raw] 
    rubs = ", ".join(rubs_L)
    pars_raw = soup0.find_all("p", {"class" : "mb-6"}) 
    pars = [p.text for p in pars_raw] 
    text = " ".join(pars)
    text = text.replace("\xa0", " ").replace("\n", " ")
    return title, author, date, diffc, rubs, text

In [36]:
get_news(news[2])

('Солнце признали солнцеподобной звездой',
 'Александр Войтюк',
 '2023-04-06',
 '4.7',
 'Астрономия',
 'Наше светило действительно мало отличается от похожих на него звезд Астрономы определили, что Солнце действительно можно считать солнцеподобной звездой, которая мало отличается от звезд, похожих на него. Ученые также выяснили, что уровень активности звезд связан с быстротой вращения и уровнем изменчивости активности. Статья опубликована в журнале Astronomy&Astrophysics. Солнцеподобными звездами считаются светила спектральных классов F, G, K, многие из которых похожи на нашу звезду. Как и Солнце, они могут проявлять значительную активность, связанную с магнитным полем, в виде темных пятен или ярких факелов в активных областях. Подобные образования влияют на изменение блеска звезды во время ее вращения, поэтому их можно отследить, анализируя фотометрические данные наблюдений, многие из которых получены во время поисков экзопланет. В результате можно получить представление о том, как за

Отлично! Теперь осталось применить функцию ко всем ссылкам в списке `news`. Чтобы сайт не понял, что мы его автоматически грабим, будем выгружать новости постепенно – с задержкой в 1.5 секунды. Импортируем для этого функцию `sleep` :

In [38]:
from time import sleep

Теперь будем применять функцию в цикле к каждой ссылке в `news`, только с одним дополнением – добавленной конструкцией `try-except`, которая позволит продолжать исполнение цикла, если при применении функции Python столкнулся с ошибкой любого вида:

In [39]:
info = []
for n in news:
    # пробуй исполнить следующий код
    try:
        res = get_news(n)
        info.append(res)
        print(n)
    # если он вызвал ошибку, печатай сообщение и иди дальше
    except:
        print("Something went wrong")
        print(n)
    sleep(1.2)

https://nplus1.ru/news/2023/04/06/pseudorabdion-longiceps
https://nplus1.ru/news/2023/04/06/in-abhorrence-dementia
https://nplus1.ru/news/2023/04/06/sun-is-sun
https://nplus1.ru/news/2023/04/06/RSVpreF
https://nplus1.ru/news/2023/04/06/muons-Napoli
https://nplus1.ru/news/2023/04/06/pseudorabdion-longiceps
https://nplus1.ru/news/2023/04/05/double-quasar-confirmed
https://nplus1.ru/news/2023/04/05/push-jab
https://nplus1.ru/news/2023/04/05/yz-ceti
https://nplus1.ru/news/2023/04/05/sonographer-vs-AI-cardiac-function
https://nplus1.ru/news/2023/04/05/methane-iron-oxidation
https://nplus1.ru/news/2023/04/05/multiple-calcified-amorphous-tumours
https://nplus1.ru/news/2023/04/05/U241
https://nplus1.ru/news/2023/04/05/quantum-mkg
https://nplus1.ru/news/2023/04/05/GLWamide-in-jellyfish
https://nplus1.ru/news/2023/04/05/record-clatch
https://nplus1.ru/news/2023/04/05/magic-monkeys
https://nplus1.ru/news/2023/04/04/jwst-new-far-galaxies
https://nplus1.ru/news/2023/04/04/elephants-self-domesticati

Посмотрим на несколько элементов `info`:

In [40]:
info[10:12]

[('Комплекс железа помог селективно окислить метан',
  'Михаил Бойм',
  '2023-04-05',
  '6.7',
  'Химия',
  'Из метана получился метанол Химики из Японии синтезировали комплекс железа с карбеновым лигандом, способный катализировать реакцию окисления алканов персульфатом натрия. Селективность реакции окисления метана до метанола составила 83 процента, а конверсия метана достигла 4,1 процента. Исследование опубликовано в Nature. Метанол (CH3OH) в промышленности получают в две стадии. Сначала при высоких температуре и давлении проводят реакцию паровой конверсии — в ней из метана (CH4) и воды образуется синтез-газ — смесь водорода и угарного газа. Затем из этой смеси напрямую получают метанол и другие органические вещества. А удобного метода получения метанола из метана напрямую до сих пор химики не придумали. Проблема разработки такого процесса заключается в том, что сам метанол окисляется намного проще, чем метан. И как только он образуется в реакционной смеси, он быстро превращается в м

Финальный штрих – импортируем `pandas` и преобразуемый полученный список кортежей в датафрейм:

In [41]:
import pandas as pd

In [42]:
df = pd.DataFrame(info)
df.head()

,0,1,2,3,4,5
0,Змея прошлась колесом в попытке убежать от гер...,Сергей Коленов,2023-04-06,2.5,Зоология,Такой защитный механизм описан у рептилий впер...
1,Активация новых нейронов гиппокампа у мышей ул...,Олег Лищук,2023-04-06,5.5,"Медицина, Биология, Психология",Гиппокампальный нейрогенез оказался перспектив...
2,Солнце признали солнцеподобной звездой,Александр Войтюк,2023-04-06,4.7,Астрономия,Наше светило действительно мало отличается от ...
3,Вакцина против респираторно-синцитиального вир...,Слава Гоменюк,2023-04-06,3.3,Медицина,Появились результаты третьей фазы испытаний В ...
4,Мюоны помогли найти погребальную камеру под ул...,Марат Хамадеев,2023-04-06,3.2,"Физика, Археология",В настоящее время в нее никак не попасть Архео...


Не везде информация сгрузилась корректно, есть проблемы унификации, можно потом поправить это, написав функцию и применив ее через `.apply()` к соответствующим столбцам.

А пока добавим содержательные названия столбцов и выгрузим датафрейм в файл:

In [43]:
df.columns = ["title", "author", "date", "diffc", "rubrics", "text"]

# плюс, изменим тип столбца со сложностью на float
df["diffc"] = df["diffc"].astype(float)

df.to_excel("nplus1.xlsx")

In [45]:
# куда Python сохранил файл

import os
os.getcwd()

'C:\\Users\\Student'

In [44]:
df

,title,author,date,diffc,rubrics,text
0,Змея прошлась колесом в попытке убежать от гер...,Сергей Коленов,2023-04-06,2.5,Зоология,Такой защитный механизм описан у рептилий впер...
1,Активация новых нейронов гиппокампа у мышей ул...,Олег Лищук,2023-04-06,5.5,"Медицина, Биология, Психология",Гиппокампальный нейрогенез оказался перспектив...
2,Солнце признали солнцеподобной звездой,Александр Войтюк,2023-04-06,4.7,Астрономия,Наше светило действительно мало отличается от ...
3,Вакцина против респираторно-синцитиального вир...,Слава Гоменюк,2023-04-06,3.3,Медицина,Появились результаты третьей фазы испытаний В ...
4,Мюоны помогли найти погребальную камеру под ул...,Марат Хамадеев,2023-04-06,3.2,"Физика, Археология",В настоящее время в нее никак не попасть Архео...
...,...,...,...,...,...,...
66,Пионный атом позволил измерить частичное восст...,Марат Хамадеев,2023-03-30,9.5,Физика,"Это подтверждает гипотезу о том, что на ранней..."
67,Физики уточнили массовый радиус протона с помо...,Марат Хамадеев,2023-03-31,9.1,Физика,"Заодно выяснив, что голографических метод извл..."
68,Физики из BESIII нашли три новых чармонийподоб...,Марат Хамадеев,2023-04-04,8.3,Физика,Но они пока не понимают природу новых резонанс...
69,Микрограммовый акустический резонатор подвергл...,Марат Хамадеев,2023-04-05,7.3,Физика,"Теперь это самый большой квантовый объект, с к..."
